In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import re
from datetime import time
from datetime import datetime 
import os

from scipy.io import readsav
from scipy import signal

from astropy.time import Time
from astropy.time import TimeDelta

from sunpy import timeseries as ts
from sunpy.net import attrs as a
from sunpy.net import hek
from sunpy.net import Fido
from sunpy.time import parse_time, find_time

import pandas as pd

import wget
import goesxrs_temp as gtem #from Ian's functions
import Onsets_temp as onsets

import warnings #Need to remove for future
warnings.filterwarnings("ignore", module="sunpy.timeseries.timeseriesbase")

# Just setup plot fonts
plt.rcParams.update({'font.size': 18,'font.family':"sans-serif",\
                         'font.sans-serif':"Arial",'mathtext.default':"regular", 'axes.linewidth' :2})

In [2]:
data_dir = r"D:\MastersProj\Data\goes15"

## Looping through all flares in directory to fetch onset info

In [3]:
#Looping through all goes-15 flares in directory
flares_df = pd.read_csv("GOES15_HEK_Data.csv")
flare_data = pd.DataFrame(columns = [])
  
onset_length = 60 #Time of fixed onset in seconds
for index, row in flares_df.iterrows():
     #Locate Flare from downloaded file
     fl_hek_start = row['event_starttime']
     fl_hek_end = row['event_endtime']
     fl_class = row['fl_goescls']
     # Printing for terminal
     print("------------------------------")
     print(f"{fl_class} Flare With start Time: {fl_hek_start}")
     try:
          flare_time=a.Time(Time(fl_hek_start, scale='utc').iso, Time(fl_hek_end,scale='utc').iso)
          day_string = flare_time.start.datetime.date().strftime("%Y%m%d")
          yr_str = flare_time.start.datetime.date().strftime("%Y")
          mnth_str = flare_time.start.datetime.date().strftime("%m")
          date_path = os.path.join(yr_str, mnth_str)
          fl_path = os.path.join(data_dir, date_path,f'*sci_gxrs-l2-irrad_g15_d*{day_string}*.nc')

          if Time(fl_hek_start).datetime.time() <= time(hour=1, minute=0, second=0):
               day_str_prev = (Time(fl_hek_start) - TimeDelta(1, format = 'jd')).datetime.date().strftime("%Y%m%d")
               fl_path_prev = os.path.join(data_dir, date_path,f'*sci_gxrs-l2-irrad_g15_d*{day_str_prev}*.nc')
               g15 = ts.TimeSeries([fl_path, fl_path_prev],concatenate=True)
          elif Time(fl_hek_start).datetime.time() >= time(hour=23, minute=0, second=0):
               day_str_post = (Time(fl_hek_start)+ TimeDelta(1, format = 'jd')).datetime.date().strftime("%Y%m%d")
               fl_path_post = os.path.join(data_dir, date_path,f'*sci_gxrs-l2-irrad_g15_d*{day_str_post}*.nc')
               g15 = ts.TimeSeries([fl_path, fl_path_post],concatenate=True)
          else:
               g15 = ts.TimeSeries(fl_path, concatenate=True)

     # Load flare into time series
          g_tims = g15.index
          g_short = g15.quantity("xrsa").value 
          g_long = g15.quantity("xrsb").value
          g_short_flag = g15.quantity("xrsa_quality").value 
          g_long_flag = g15.quantity("xrsb_quality").value 

          # Create a Boolean mask where any flags are located
          mask_long = g_long_flag != 0
          mask_short = g_short_flag != 0

          # Set the corresponding values in g_short and g_long to NaN
          g_short[mask_short] = np.nan
          g_long[mask_long] = np.nan

          # Set sub-zero values not picked up by flags to nans
          mask_short_mask = g_short <= 0 
          g_short[mask_short_mask] = np.nan 

          #Load into dataframes
          df_long = pd.Series(g_long, index = pd.DatetimeIndex(g15.index))
          df_short = pd.Series(g_short, index = pd.DatetimeIndex(g15.index))

     #Calculating time of non-backsubbed peak (for 2s cadence instead of 1-min)
          bcktrunc_18=df_long.truncate(flare_time.start.iso,flare_time.end.iso)
          old_peak = bcktrunc_18.idxmax()

          print("File Read in!")

     # Background Time Calculation Function
          #bckt, bcktrunc_054, bcktrunc_18, bckdata = background_2(df_short, df_long, start_time = Time(row['Start Time']), peak_time = Time(row['Peak Time']))
          bck_startt, bck_endt, _, _, _, bck_flag = onsets.background(df_short, df_long, start_time = Time(fl_hek_start), peak_time = old_peak)
          srch_time = a.Time(Time(bck_startt, scale='utc').iso, Time(bck_endt,scale='utc').iso)
          trunc_054 = df_short.truncate(srch_time.start.iso,srch_time.end.iso)
          trunc_18 = df_long.truncate(srch_time.start.iso,srch_time.end.iso)
          bck_t = trunc_054.index
          bck_short = np.mean(trunc_054)
          bck_long = np.mean(trunc_18)
          bck_short_std = np.std(trunc_054)
          bck_long_std = np.std(trunc_18)

     # Background Subtracting the Data
          short_backsub = df_short - bck_short
          long_backsub = df_long - bck_long

          print("Backsub worked!")

     # Calculating true peak time (using backsubbed long channel flux)
          flare_long = long_backsub.truncate(flare_time.start.iso,flare_time.end.iso)
          long_peakt = flare_long.idxmax()
          flare_short = short_backsub.truncate(flare_time.start.iso,flare_time.end.iso)
          short_peakt = flare_short.idxmax()

          long_peakfl_18 = flare_long[long_peakt]
          long_peakfl_054 = flare_short[long_peakt]
          short_peakfl_18 = flare_long[short_peakt]
          short_peakfl_054 = flare_short[short_peakt]

          true_peak = long_peakt #Varaible change for old code

     # Onset and TEM Calculations of different onset definitions
          #onset_start = (Time(bck_endt) + (Time(true_peak)-Time(bck_endt))/8).datetime
          srch_start = Time(bck_endt) - TimeDelta(60, format = 'sec')
          trange_search = a.Time(srch_start, true_peak)
          g_long_srch = long_backsub.truncate(trange_search.start.iso,trange_search.end.iso)
          g_short_srch = short_backsub.truncate(trange_search.start.iso,trange_search.end.iso)

          indexes = g_long_srch[(g_long_srch >= 4*bck_long_std) & (g_short_srch >= 4* bck_short_std)].index
          if len(indexes) == 0:
               raise ValueError
          else:
               first_index = indexes[0]
     
     # Deciding what to use as the first point after the background
          if first_index > bck_endt:
               onset_start = first_index
          else:
               onset_start = bck_endt

          endt_eighth = onsets.scaled_onset(onset_start, true_peak, 8)
          timedelta_eighth = pd.to_timedelta(endt_eighth - onset_start).total_seconds()
          endt_sixth = onsets.scaled_onset(onset_start, true_peak, 6)
          timedelta_sixth = pd.to_timedelta(endt_sixth - onset_start).total_seconds()
          endt_quarter = onsets.scaled_onset(onset_start, true_peak, 4)
          timedelta_quarter = pd.to_timedelta(endt_quarter - onset_start).total_seconds()
          endt_third = onsets.scaled_onset(onset_start, true_peak, 3)
          timedelta_third = pd.to_timedelta(endt_third - onset_start).total_seconds()
          endt_half = onsets.scaled_onset(onset_start, true_peak, 2)
          timedelta_half = pd.to_timedelta(endt_half - onset_start).total_seconds()
          endt_twothirds = onsets.scaled_onset(onset_start, true_peak, 3/2)
          timedelta_twothirds = pd.to_timedelta(endt_twothirds - onset_start).total_seconds()
          endt_threequart = onsets.scaled_onset(onset_start, true_peak, 4/3)
          timedelta_threequart= pd.to_timedelta(endt_threequart - onset_start).total_seconds()

          print("Calculated onset times!")
          
          if onset_start < endt_eighth:
               tmk_eighth, em_eighth, tmk_upper_eighth, tmk_lower_eighth, em_err_eighth  = onsets.onset_tem(df_short, df_long, short_backsub, long_backsub, onset_start, endt_eighth, bck_short_std, bck_long_std)
          else:
               tmk_eighth = np.nan
               em_eighth = np.nan
          if onset_start < endt_sixth:
               tmk_sixth, em_sixth, tmk_upper_sixth, tmk_lower_sixth, em_err_sixth  = onsets.onset_tem(df_short, df_long, short_backsub, long_backsub, onset_start, endt_sixth, bck_short_std, bck_long_std)
          else:
               tmk_sixth = np.nan
               em_sixth = np.nan
          if onset_start < endt_quarter:
               tmk_quarter, em_quarter, tmk_upper_quarter, tmk_lower_quarter, em_err_quarter  = onsets.onset_tem(df_short, df_long, short_backsub, long_backsub, onset_start, endt_quarter, bck_short_std, bck_long_std)
          else:
               tmk_quarter = np.nan
               em_quarter = np.nan
          if onset_start < endt_third:
               tmk_third, em_third, tmk_upper_third, tmk_lower_third, em_err_third  = onsets.onset_tem(df_short, df_long, short_backsub, long_backsub, onset_start, endt_third, bck_short_std, bck_long_std)
          else:
               tmk_third = np.nan
               em_third = np.nan
          if onset_start < endt_half:
               tmk_half, em_half, tmk_upper_half, tmk_lower_half, em_err_half = onsets.onset_tem(df_short, df_long, short_backsub, long_backsub, onset_start, endt_half, bck_short_std, bck_long_std)
          else:
               tmk_half = np.nan
               em_half = np.nan

          
     # Calculating teperature at flare peak (long channel)
          trunc_054_tem_peakl = short_backsub[true_peak]
          trunc_18_tem_peakl = long_backsub[true_peak]
             
          tmk_peakl, em_peakl = gtem.get_tem_old(trunc_18_tem_peakl, trunc_054_tem_peakl)
     
     #Calculating temperature at flare peak (short_channel)
          trunc_054_tem_peaks = short_backsub[short_peakt]
          trunc_18_tem_peaks = long_backsub[short_peakt]
          tmk_peaks, em_peaks = gtem.get_tem_old(trunc_18_tem_peaks, trunc_054_tem_peaks)

     # Calculating max flare temperature
          trunc_054_tem_max = short_backsub.truncate(endt_half, Time(true_peak).datetime)
          trunc_18_tem_max = long_backsub.truncate(endt_half, Time(true_peak).datetime)
          tmk_maxx, em_maxx = gtem.get_tem_old(trunc_18_tem_max, trunc_054_tem_max)
          try:
               tmk_max = np.nanmax(tmk_maxx)
               tmk_max_index = np.nanargmax(tmk_maxx) 
               em_max = em_maxx[tmk_max_index]
          except:
               tmk_max = np.nan
               em_max = np.nan
          
          print("SUCCESS!!!")
          
          filenotfound_flag = False
          index_flag_err = False

          flare_data = pd.concat([flare_data, pd.DataFrame({'Peak Time Long': [true_peak], 'Peak Time Short':[short_peakt],'Peak Flux': [long_peakfl_18], 'Background Start Time':[bck_startt], 'Background End Time': [bck_endt], 'Onset Start Time':[onset_start], 
                                                            'Peak Temp Long': [tmk_peakl], 'Peak EM Long': [em_peakl],'Peak Temp Short': [tmk_peaks], 'Peak EM Short': [em_peaks], 'Flare Max Temp': [tmk_max], 'Flare Max EM': [em_max], 
                                                            'Temp 1/8': [tmk_eighth], 'Temp 1/8 Upper': [tmk_upper_eighth], 'Temp 1/8 Lower': [tmk_lower_eighth],'EM 1/8': [em_eighth], 'EM 1/8 Error': [em_err_eighth], 'Tdelta 1/8': [timedelta_eighth],
                                                            'Temp 1/6': [tmk_sixth], 'Temp 1/6 Upper': [tmk_upper_sixth], 'Temp 1/6 Lower': [tmk_lower_sixth], 'EM 1/6': [em_sixth], 'EM 1/6 Error': [em_err_sixth], 'Tdelta 1/6': [timedelta_sixth],
                                                            'Temp 1/4': [tmk_quarter], 'Temp 1/4 Upper': [tmk_upper_quarter], 'Temp 1/4 Lower': [tmk_lower_quarter],'EM 1/4': [em_quarter], 'EM 1/4 Error': [em_err_quarter], 'Tdelta 1/4': [timedelta_quarter],
                                                            'Temp 1/3': [tmk_third], 'Temp 1/3 Upper': [tmk_upper_third], 'Temp 1/3 Lower': [tmk_lower_third], 'EM 1/3': [em_third], 'EM 1/3 Error': [em_err_third],'Tdelta 1/3':[timedelta_third],
                                                            'Temp 1/2': [tmk_half], 'Temp 1/2 Upper': [tmk_upper_half], 'Temp 1/2 Lower': [tmk_lower_half], 'EM 1/2': [em_half], 'EM 1/2 Error': [em_err_half], 'Tdelta 1/2': [timedelta_half],
                                                            'Background Flag': [bck_flag], 'FileNotFound Flag':[filenotfound_flag], 'IndexError Flag': [index_flag_err]})], ignore_index=True)


     except ValueError:
          filenotfound_flag = True
          index_flag_err = False
          print("NO FILE FOUND")
          print("ERROR, NAN ROW PLACED")
          print("---------------------------------------")
          flare_data = pd.concat([flare_data, pd.DataFrame({'Peak Time Long': [np.nan], 'Peak Time Short':[np.nan],'Peak Flux': [np.nan], 'Background Start Time':[np.nan], 'Background End Time': [np.nan], 'Onset Start Time':[np.nan], 
                                                                'Peak Temp Long': [np.nan], 'Peak EM Long': [np.nan],'Peak Temp Short': [np.nan], 'Peak EM Short': [np.nan], 'Flare Max Temp': [np.nan], 'Flare Max EM': [np.nan], 
                                                                 'Temp 1/8': [np.nan], 'Temp 1/8 Upper': [np.nan], 'Temp 1/8 Lower': [np.nan],'EM 1/8': [np.nan], 'EM 1/8 Error': [np.nan],'Tdelta 1/8': [np.nan],
                                                                 'Temp 1/6': [np.nan], 'Temp 1/6 Upper': [np.nan], 'Temp 1/6 Lower': [np.nan], 'EM 1/6': [np.nan], 'EM 1/6 Error': [np.nan], 'Tdelta 1/6': [np.nan],
                                                                 'Temp 1/4': [np.nan], 'Temp 1/4 Upper': [np.nan], 'Temp 1/4 Lower': [np.nan],'EM 1/4': [np.nan], 'EM 1/4 Error': [np.nan], 'Tdelta 1/4': [np.nan],
                                                                 'Temp 1/3': [np.nan], 'Temp 1/3 Upper': [np.nan], 'Temp 1/3 Lower': [np.nan], 'EM 1/3': [np.nan], 'EM 1/3 Error': [np.nan], 'Tdelta 1/3':[np.nan],
                                                                 'Temp 1/2': [np.nan], 'Temp 1/2 Upper': [np.nan], 'Temp 1/2 Lower': [np.nan], 'EM 1/2': [np.nan], 'EM 1/2 Error': [np.nan], 'Tdelta 1/2': [np.nan],
                                                                 'Background Flag': [np.nan], 'FileNotFound Flag':[filenotfound_flag], 'IndexError Flag': [index_flag_err]})], ignore_index=True)
          continue

     except IndexError:
          filenotfound_flag = False
          index_flag_err = True
          print("ERROR, NAN ROW PLACED")
          print("---------------------------------------")
          flare_data = pd.concat([flare_data, pd.DataFrame({'Peak Time Long': [np.nan], 'Peak Time Short':[np.nan],'Peak Flux': [np.nan], 'Background Start Time':[np.nan], 'Background End Time': [np.nan], 'Onset Start Time':[np.nan], 
                                                                'Peak Temp Long': [np.nan], 'Peak EM Long': [np.nan],'Peak Temp Short': [np.nan], 'Peak EM Short': [np.nan], 'Flare Max Temp': [np.nan], 'Flare Max EM': [np.nan], 
                                                                 'Temp 1/8': [np.nan], 'Temp 1/8 Upper': [np.nan], 'Temp 1/8 Lower': [np.nan],'EM 1/8': [np.nan], 'EM 1/8 Error': [np.nan],'Tdelta 1/8': [np.nan],
                                                                 'Temp 1/6': [np.nan], 'Temp 1/6 Upper': [np.nan], 'Temp 1/6 Lower': [np.nan], 'EM 1/6': [np.nan], 'EM 1/6 Error': [np.nan], 'Tdelta 1/6': [np.nan],
                                                                 'Temp 1/4': [np.nan], 'Temp 1/4 Upper': [np.nan], 'Temp 1/4 Lower': [np.nan],'EM 1/4': [np.nan], 'EM 1/4 Error': [np.nan], 'Tdelta 1/4': [np.nan],
                                                                 'Temp 1/3': [np.nan], 'Temp 1/3 Upper': [np.nan], 'Temp 1/3 Lower': [np.nan], 'EM 1/3': [np.nan], 'EM 1/3 Error': [np.nan], 'Tdelta 1/3':[np.nan],
                                                                 'Temp 1/2': [np.nan], 'Temp 1/2 Upper': [np.nan], 'Temp 1/2 Lower': [np.nan], 'EM 1/2': [np.nan], 'EM 1/2 Error': [np.nan], 'Tdelta 1/2': [np.nan],
                                                                 'Background Flag': [np.nan], 'FileNotFound Flag':[filenotfound_flag], 'IndexError Flag': [index_flag_err]})], ignore_index=True)
          continue
     except:
          filenotfound_flag = False
          index_flag_err = False
          print("ERROR, NAN ROW PLACED")
          print("---------------------------------------")
          flare_data = pd.concat([flare_data, pd.DataFrame({'Peak Time Long': [np.nan], 'Peak Time Short':[np.nan],'Peak Flux': [np.nan], 'Background Start Time':[np.nan], 'Background End Time': [np.nan], 'Onset Start Time':[np.nan], 
                                                                'Peak Temp Long': [np.nan], 'Peak EM Long': [np.nan],'Peak Temp Short': [np.nan], 'Peak EM Short': [np.nan], 'Flare Max Temp': [np.nan], 'Flare Max EM': [np.nan], 
                                                                 'Temp 1/8': [np.nan], 'Temp 1/8 Upper': [np.nan], 'Temp 1/8 Lower': [np.nan],'EM 1/8': [np.nan], 'EM 1/8 Error': [np.nan],'Tdelta 1/8': [np.nan],
                                                                 'Temp 1/6': [np.nan], 'Temp 1/6 Upper': [np.nan], 'Temp 1/6 Lower': [np.nan], 'EM 1/6': [np.nan], 'EM 1/6 Error': [np.nan], 'Tdelta 1/6': [np.nan],
                                                                 'Temp 1/4': [np.nan], 'Temp 1/4 Upper': [np.nan], 'Temp 1/4 Lower': [np.nan],'EM 1/4': [np.nan], 'EM 1/4 Error': [np.nan], 'Tdelta 1/4': [np.nan],
                                                                 'Temp 1/3': [np.nan], 'Temp 1/3 Upper': [np.nan], 'Temp 1/3 Lower': [np.nan], 'EM 1/3': [np.nan], 'EM 1/3 Error': [np.nan], 'Tdelta 1/3':[np.nan],
                                                                 'Temp 1/2': [np.nan], 'Temp 1/2 Upper': [np.nan], 'Temp 1/2 Lower': [np.nan], 'EM 1/2': [np.nan], 'EM 1/2 Error': [np.nan], 'Tdelta 1/2': [np.nan],
                                                                 'Background Flag': [np.nan], 'FileNotFound Flag':[filenotfound_flag], 'IndexError Flag': [index_flag_err]})], ignore_index=True)
          continue


if len(flare_data) == len(flares_df):
     full_data = pd.concat([flares_df, flare_data],axis = 1)
else:
     print("ERROR, data lengths do not match")

full_data.to_csv("Full_Run_v8.csv")

------------------------------
B1.7 Flare With start Time: 2010-04-02 23:57:00.000
File Read in!
NO FILE FOUND
ERROR, NAN ROW PLACED
---------------------------------------
------------------------------
B7.4 Flare With start Time: 2010-04-03 09:04:00.000
File Read in!
NO FILE FOUND
ERROR, NAN ROW PLACED
---------------------------------------
------------------------------
B3.0 Flare With start Time: 2010-04-03 09:04:00.000
File Read in!
NO FILE FOUND
ERROR, NAN ROW PLACED
---------------------------------------
------------------------------
B1.4 Flare With start Time: 2010-04-05 00:43:00.000
File Read in!
NO FILE FOUND
ERROR, NAN ROW PLACED
---------------------------------------
------------------------------
B1.1 Flare With start Time: 2010-04-05 08:45:00.000
File Read in!
NO FILE FOUND
ERROR, NAN ROW PLACED
---------------------------------------
------------------------------
B1.3 Flare With start Time: 2010-04-05 11:19:00.000
File Read in!
NO FILE FOUND
ERROR, NAN ROW PLACED
--

C:\Users\William\AppData\Local\Temp\ipykernel_4780\4107225266.py:217: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  flare_data = pd.concat([flare_data, pd.DataFrame({'Peak Time Long': [true_peak], 'Peak Time Short':[short_peakt],'Peak Flux': [long_peakfl_18], 'Background Start Time':[bck_startt], 'Background End Time': [bck_endt], 'Onset Start Time':[onset_start],


File Read in!
Backsub worked!
Calculated onset times!
SUCCESS!!!
------------------------------
B1.0 Flare With start Time: 2010-04-09 20:18:00.000
File Read in!
Backsub worked!
NO FILE FOUND
ERROR, NAN ROW PLACED
---------------------------------------
------------------------------
B1.0 Flare With start Time: 2010-04-09 20:32:00.000
File Read in!
Backsub worked!
Calculated onset times!
SUCCESS!!!
------------------------------
B3.7 Flare With start Time: 2010-04-13 04:39:00.000
File Read in!
Backsub worked!
ERROR, NAN ROW PLACED
---------------------------------------
------------------------------
B2.2 Flare With start Time: 2010-04-17 04:30:00.000
File Read in!
Backsub worked!
Calculated onset times!
SUCCESS!!!
------------------------------
B2.1 Flare With start Time: 2010-04-17 04:37:00.000
File Read in!
Backsub worked!
Calculated onset times!
SUCCESS!!!
------------------------------
B1.3 Flare With start Time: 2010-04-17 21:34:00.000
File Read in!
Backsub worked!
Calculated ons

C:\Users\William\AppData\Local\Temp\ipykernel_4780\4107225266.py:205: RuntimeWarning: All-NaN slice encountered
  tmk_max = np.nanmax(tmk_maxx)


File Read in!
Backsub worked!
ERROR, NAN ROW PLACED
---------------------------------------
------------------------------
B1.1 Flare With start Time: 2010-04-24 22:56:00.000
File Read in!
Backsub worked!
NO FILE FOUND
ERROR, NAN ROW PLACED
---------------------------------------
------------------------------
B1.1 Flare With start Time: 2010-04-24 22:56:00.000
File Read in!
Backsub worked!
NO FILE FOUND
ERROR, NAN ROW PLACED
---------------------------------------
------------------------------
B1.8 Flare With start Time: 2010-04-30 00:17:00.000
NO FILE FOUND
ERROR, NAN ROW PLACED
---------------------------------------
------------------------------
C2.2 Flare With start Time: 2010-04-30 19:28:00.000
NO FILE FOUND
ERROR, NAN ROW PLACED
---------------------------------------
------------------------------
C5.7 Flare With start Time: 2010-05-01 01:34:00.000
NO FILE FOUND
ERROR, NAN ROW PLACED
---------------------------------------
------------------------------
B1.6 Flare With start 

...

SUCCESS!!!
------------------------------
A9.6 Flare With start Time: 2019-12-18 11:39:00.000
File Read in!
Backsub worked!
Calculated onset times!
NO FILE FOUND
ERROR, NAN ROW PLACED
---------------------------------------
------------------------------
A9.5 Flare With start Time: 2019-12-19 04:56:00.000
File Read in!
Backsub worked!
Calculated onset times!
SUCCESS!!!
------------------------------
A9.7 Flare With start Time: 2019-12-19 07:24:00.000
File Read in!
Backsub worked!
Calculated onset times!
SUCCESS!!!
------------------------------
A8.7 Flare With start Time: 2019-12-20 10:54:00.000
File Read in!
Backsub worked!
NO FILE FOUND
ERROR, NAN ROW PLACED
---------------------------------------
------------------------------
B1.5 Flare With start Time: 2019-12-23 21:46:00.000
File Read in!
Backsub worked!
NO FILE FOUND
ERROR, NAN ROW PLACED
---------------------------------------
------------------------------
B1.1 Flare With start Time: 2019-12-24 05:24:00.000
File Read in!
Backs

C:\Users\William\AppData\Local\Temp\ipykernel_4780\4107225266.py:205: RuntimeWarning: All-NaN slice encountered
  tmk_max = np.nanmax(tmk_maxx)


File Read in!
Backsub worked!
Calculated onset times!


C:\Users\William\AppData\Local\Temp\ipykernel_4780\4107225266.py:205: RuntimeWarning: All-NaN slice encountered
  tmk_max = np.nanmax(tmk_maxx)


SUCCESS!!!
------------------------------
B1.3 Flare With start Time: 2020-01-09 19:42:00.000
File Read in!
Backsub worked!
NO FILE FOUND
ERROR, NAN ROW PLACED
---------------------------------------
------------------------------
B1.1 Flare With start Time: 2020-01-10 11:52:00.000
File Read in!
Backsub worked!
Calculated onset times!
SUCCESS!!!
------------------------------
B1.6 Flare With start Time: 2020-01-10 11:55:00.000
File Read in!
Backsub worked!
NO FILE FOUND
ERROR, NAN ROW PLACED
---------------------------------------
------------------------------
B1.1 Flare With start Time: 2020-01-10 14:41:00.000
File Read in!
Backsub worked!
Calculated onset times!
SUCCESS!!!
------------------------------
B1.6 Flare With start Time: 2020-01-10 14:41:00.000
File Read in!
Backsub worked!
Calculated onset times!
SUCCESS!!!
------------------------------
B1.0 Flare With start Time: 2020-01-11 15:23:00.000
File Read in!
Backsub worked!
NO FILE FOUND
ERROR, NAN ROW PLACED
------------------